In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os,sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import helper
import simulation
import time
import cv2
import skimage.io
from skimage import color, util

video_dir = "./"
video_name = "Square"
video_extension = ".mp4"
frames_dir = os.path.join(video_dir, "frames_" + video_name)

if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

In [2]:
import torch
import torch.nn as nn
import pytorch_unet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = pytorch_unet.UNet(6)
model = model.to(device)

weights_path = "./unet-figures.pt"
model.load_state_dict(torch.load(weights_path))
model.eval()

UNet(
  (dconv_down1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_si

In [3]:
import torchvision.utils
from torchvision import transforms

def reverse_transform(inp):
    inp = inp.transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    inp = (inp * 255).astype(np.uint8)
    
    return inp

In [4]:
save_frames = False

vidcap = cv2.VideoCapture(os.path.join(video_dir, video_name + video_extension))
success, image = vidcap.read()
frame_amount = 0
start_time = time.time()
resize_size = 192
while success:
    # preprocess
    image = util.invert(image.astype('uint8'))    
    if len(image.shape) > 2:
        image = color.rgb2gray(image)
    image = skimage.transform.resize(image, (resize_size, resize_size), preserve_range=True, anti_aliasing=True)
    image = np.reshape(image, [*image.shape, 1])
    image = np.concatenate((image, image, image), axis=2)
        
    image -= image.min()
    image /= image.max()
        
    image[image > 0.15] = 1
    image = image.round().astype(np.float32)
    
    # unet in da house
    model.eval()   # Set model to evaluate mode

    image = image.transpose((2, 0, 1))
    tensor = torch.from_numpy(image).to(torch.float32)
    tensor = tensor.reshape(1, *tensor.shape)
    pred = model(tensor.to(device=device))

    pred = pred.data.cpu().numpy()
    pred_rgb = [helper.masks_to_colorimg(x) for x in pred]
    
    # save frame
    image = image.transpose((1, 2, 0))
    image *= 255
    
    if save_frames is True:
        cv2.imwrite(os.path.join(frames_dir, video_name + "_frame%d.jpg" % frame_amount), image)      
    success, image = vidcap.read()
    frame_amount += 1
#     print('Read a new frame: ', frame_amount, end='\r')

elapsed_time = time.time() - start_time
print("Elapsed time", elapsed_time)

Elapsed time 127.34195685386658


In [5]:
fps = frame_amount / elapsed_time
print(fps)

4.429019420890695
